In [1]:
!nvidia-smi

Thu Jun 20 16:16:38 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.40.04    Driver Version: 418.40.04    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           Off  | 00000000:04:00.0 Off |                  Off |
| N/A   29C    P8     9W / 250W |      0MiB / 24451MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P40           Off  | 00000000:06:00.0 Off |                  Off |
| N/A   47C    P0   104W / 250W |   4279MiB / 24451MiB |     74%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [2]:
CUDA_VISIBLE_DEVICES = '0,2,3'

In [3]:
%load_ext autoreload
%autoreload 2

import os

import __init_path__
import env
import mlflow
from odometry.utils import make_memory_safe
from odometry.preprocessing import parsers, estimators, prepare_trajectory
from odometry.data_manager import GeneratorFactory


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



## Prepare dataset

In [4]:
def initialize_parser():
    trajectory_dir = 'discoman'
    json_path = os.path.join(env.DATASET_PATH, 'renderbox/iros2019/dset/output/deprecated/000001/0_traj.json')
    parser = parsers.DISCOMANParser(trajectory_dir, json_path)
    return parser

In [5]:
def initialize_estimators(target_size):
    quaternion2euler_estimator = estimators.Quaternion2EulerEstimator(input_col=['q_w', 'q_x', 'q_y', 'q_z'],
                                                                      output_col=['euler_x', 'euler_y', 'euler_z'])
    
    depth_checkpoint = os.path.abspath('../weights/model-199160')
    struct2depth_estimator = estimators.Struct2DepthEstimator(input_col='path_to_rgb',
                                                              output_col='path_to_depth',
                                                              sub_dir='depth',
                                                              checkpoint=depth_checkpoint, 
                                                              height=target_size[0],
                                                              width=target_size[1])
    
    cols = ['euler_x', 'euler_y', 'euler_z', 't_x', 't_y', 't_z']
    input_col = cols + [col + '_next' for col in cols]
    output_col = cols
    global2relative_estimator = estimators.Global2RelativeEstimator(input_col=input_col,
                                                                    output_col=output_col)
    
    optical_flow_checkpoint = '/Vol0/user/f.konokhov/tfoptflow/tfoptflow/tmp/pwcnet.ckpt-84000'
    #optical_flow_checkpoint = os.path.abspath(../weights/pwcnet.ckpt-595000')  # official weights
    pwcnet_estimator = estimators.PWCNetEstimator(input_col=['path_to_rgb', 'path_to_rgb_next'],
                                                  output_col='path_to_optical_flow',
                                                  sub_dir='optical_flow',
                                                  checkpoint=optical_flow_checkpoint)
    
    single_frame_estimators = [quaternion2euler_estimator, struct2depth_estimator]
    pair_frames_estimators = [global2relative_estimator, pwcnet_estimator]
    return single_frame_estimators, pair_frames_estimators

In [6]:
def prepare_dataset(trajectory_names, target_size):
    parser = initialize_parser()
    single_frame_estimators, pair_frames_estimators = initialize_estimators(target_size)
    for trajectory_name in trajectory_names:
        trajectory_dir = 'discoman/{}'.format(trajectory_name)
        df = prepare_trajectory(trajectory_dir, 
                                parser=parser, 
                                single_frame_estimators=single_frame_estimators,
                                pair_frames_estimators=pair_frames_estimators,
                                stride=1)
        df.to_csv(os.path.join(trajectory_dir, 'df.csv'), index=False)

In [7]:
# toy example
config = { 
    "train_sequences": [
        "train/0085",
    ],
    "val_sequences": [
        "val/0007"
    ]
}

target_size = (120, 160)

In [8]:
!rm -r discoman

In [9]:
import itertools

make_memory_safe(prepare_dataset, cuda_visible_devices=CUDA_VISIBLE_DEVICES)(
    itertools.chain(config['train_sequences'], config['val_sequences']),
    target_size=target_size
)   



Computation settings:



Using TensorFlow backend.


CUDA visible devices:     0,2,3
Available GPUs:           /job:localhost/replica:0/task:0/device:GPU:0, /job:localhost/replica:0/task:0/device:GPU:1, /job:localhost/replica:0/task:0/device:GPU:2
Allow growth:             True
GPU memory fraction:      0.33
Number of CPU:            8
Number of CPU threads:    16

Random seed:              42



Instructions for updating:
Colocations handled automatically by placer.


W0620 16:19:02.653527 47083014344320 deprecation.py:323] From /Vol0/user/a.vorontsova/miniconda3/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use keras.layers.batch_normalization instead.


...

W0620 16:19:17.142702 47083014344320 deprecation.py:323] From /Vol0/user/a.vorontsova/odometry/submodules/tfoptflow/tfoptflow/model_pwcnet.py:1300: conv2d_transpose (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.conv2d_transpose instead.


... model built.
Loading model checkpoint /Vol0/user/f.konokhov/tfoptflow/tfoptflow/tmp/pwcnet.ckpt-84000 for eval or testing...

INFO:tensorflow:Restoring parameters from /Vol0/user/f.konokhov/tfoptflow/tfoptflow/tmp/pwcnet.ckpt-84000


I0620 16:19:21.677202 47083014344320 saver.py:1270] Restoring parameters from /Vol0/user/f.konokhov/tfoptflow/tfoptflow/tmp/pwcnet.ckpt-84000


... model loaded


PWCNet              : 100%|██████████| 2130/2130 [02:11<00:00, 16.21it/s]


In [10]:
!ls discoman

train  val


## Odometry

In [11]:
dataset = GeneratorFactory(
    csv_name='df.csv',
    dataset_root='discoman',
    train_sequences=config['train_sequences'],
    val_sequences=config['val_sequences'],
    target_size=target_size,
    x_col=['path_to_optical_flow'],
    y_col=['euler_x', 'euler_y', 'euler_z', 't_x', 't_y', 't_z'],
    image_columns=['path_to_optical_flow'],
    load_modes=['flow_xy'],
    preprocess_modes=['flow_xy'],
    val_sampling_step=2,
    cached_imgs={}
)

100%|██████████| 1/1 [00:00<00:00, 34.59it/s]

Set empty cache


In [12]:
dataset.input_shapes

[(120, 160, 2)]

In [13]:
dataset.df_val.head()

,path_to_depth,path_to_rgb,timestamp,path_to_depth_next,path_to_rgb_next,timestamp_next,euler_x,euler_y,euler_z,t_x,t_y,t_z,path_to_optical_flow,trajectory_id
0,depth/0.npy,rgb/0.jpg,0,depth/1.npy,rgb/1.jpg,5,0.000028,-0.000698,4.821875e-07,-0.000002,-0.000005,0.000244,val/0007/optical_flow/0_1.npy,val/0007
2,depth/2.npy,rgb/2.jpg,10,depth/3.npy,rgb/3.jpg,15,0.000166,-0.004189,5.193066e-05,-0.000038,-0.000020,0.001501,val/0007/optical_flow/2_3.npy,val/0007
4,depth/4.npy,rgb/4.jpg,20,depth/5.npy,rgb/5.jpg,25,0.000546,-0.007674,-3.795352e-04,0.000367,-0.000097,0.002686,val/0007/optical_flow/4_5.npy,val/0007
6,depth/6.npy,rgb/6.jpg,30,depth/7.npy,rgb/7.jpg,35,0.000459,-0.011167,-1.948336e-04,0.000145,-0.000071,0.003850,val/0007/optical_flow/6_7.npy,val/0007
8,depth/8.npy,rgb/8.jpg,40,depth/9.npy,rgb/9.jpg,45,0.000588,-0.014653,-6.542083e-04,0.000454,-0.000208,0.004835,val/0007/optical_flow/8_9.npy,val/0007


In [15]:
def train(dataset):
    import numpy as np
    import pandas as pd
    from functools import partial
    
    from odometry.models import ModelFactory, construct_flexible_model
    from odometry.linalg import RelativeTrajectory
    from odometry.evaluation import calculate_metrics
    from odometry.utils import visualize_trajectory, visualize_trajectory_with_gt
    
    
    construct_graph_fn = partial(construct_flexible_model, use_gated_convolutions=False)
    model_factory = ModelFactory(
        construct_graph_fn,
        input_shapes=dataset.input_shapes,
        lr=0.001,
        loss='mae',
        scale_rotation=50
    )
    model = model_factory.construct()

    print('TRAIN')
    train_generator = dataset.get_train_generator()
    model.fit_generator(train_generator, steps_per_epoch=len(train_generator), epochs=1)
    
    print('PREDICT')
    val_generator = dataset.get_val_generator()
    model_output = model.predict_generator(val_generator, steps=len(val_generator))
    predictions = pd.DataFrame(data=np.concatenate(model_output, 1), 
                               index=dataset.df_val.index,
                               columns=dataset.y_col)
    
    print('EVALUATE')
    for trajectory_id, indices in dataset.df_val.groupby(by='trajectory_id').indices.items():
        trajectory_id = trajectory_id.replace('/', '_')

        gt_trajectory = RelativeTrajectory.from_dataframe(dataset.df_val.iloc[indices]).to_global()
        predicted_trajectory = RelativeTrajectory.from_dataframe(predictions.iloc[indices]).to_global()

        predicted_trajectory.plot('plot_{}.html'.format(trajectory_id))

        metrics = calculate_metrics(gt_trajectory, predicted_trajectory)
        print(metrics)
        
        title = '{}: {}'.format(trajectory_id.upper(), metrics)
        visualize_trajectory(predicted_trajectory, title=title, is_3d=True,
                             file_name='visualize_3d_{}.html'.format(trajectory_id))
        visualize_trajectory(predicted_trajectory, title=title, is_3d=False,
                             file_name='visualize_2d_{}.html'.format(trajectory_id))

        visualize_trajectory_with_gt(gt_trajectory, predicted_trajectory, title=title, is_3d=True,
                                     file_name='visualize_3d_with_gt_{}.html'.format(trajectory_id))
        visualize_trajectory_with_gt(gt_trajectory, predicted_trajectory, title=title, is_3d=False,
                                     file_name='visualize_2d_with_gt_{}.html'.format(trajectory_id))
    
mlflow.set_experiment("pipeline")
with mlflow.start_run(run_name="basic_setup"):
    mlflow.log_param("val_sequences", config["val_sequences"])
    train(dataset)
#make_memory_safe(train, cuda_visible_devices=CUDA_VISIBLE_DEVICES)(dataset) # mysteriously fails with pyquaternion sometimes   

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


W0620 16:33:07.225721 47083014344320 deprecation.py:323] From /Vol0/user/a.vorontsova/miniconda3/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


TRAIN
Set empty cache
Instructions for updating:
Use tf.cast instead.


W0620 16:33:07.795037 47083014344320 deprecation.py:323] From /Vol0/user/a.vorontsova/miniconda3/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Epoch 1/1
17/17 [==============================] - 36s 2s/step - loss: 1.1876 - euler_x_loss: 0.0044 - euler_y_loss: 0.0136 - euler_z_loss: 0.0053 - t_x_loss: 0.0039 - t_y_loss: 0.0059 - t_z_loss: 0.0112 - euler_x_rmse: 0.0044 - euler_y_rmse: 0.0136 - euler_z_rmse: 0.0053 - t_x_rmse: 0.0040 - t_y_rmse: 0.0059 - t_z_rmse: 0.0112
PREDICT
EVALUATE
{'ATE': 8.057516130349205, 'RMSE_t': 10.31881839916596, 'RMSE_r': 2.3203031168905704e-06, 'RPE_t': 4362245.302380594, 'RPE_r': 0.7713171937197248, 'RPE_divider': 567645}
